## Union-find data structure

- union-find data structure라고 불린다
- 몇몇의 disjoin subset으로 모든 element를 가지고 있는 데이터 구조
- 세가지 연산 : union, find, makeset
- linked list를 통해 표현되지만 종종 tree like 구조를 통해 구현된다.
- Kruskal 알고리즘에서는 두개의 vertices가 같은 set인지를 판별하는데 사용되고 O(1) 이다.

### PROBLEM : tree like structure can become unbalanced

1. Union by rank : 작은 차수의 sub-tree를 큰 차수의 sub-tree에 붙인다. 적어도 큰 차수의 sub-tree만큼의 차수가 보장됨
2. path compression : 모든 element를 flat하고 바로 root에 붙도록 한다. O(1) 을 얻을 수 있다. find 함수를 호출할때... tree가 flat된다.

## Spanning Trees


- spanning tree란 connected, acyclic, undirected graph
- 일반적으로 tree는 여러개의 spanning tree를 가질 수 있다.
- edge에는 weight가 부여되어 있고 minimum spanning tree란 edge의 wieght의 합이 최소인 tree
- k-means 클러스터링, optimization등에 많이 사용된다
- 표준적인 알고리즘으로 : prim(노드중심), kruskal(엣지중심) 등이 있다.


- 먼저 모든 edge를 weight에 따라 sorting한다.
- sorting은 mergesort, quicksort에 의해 O(nlogn) 으로 가능하다.
- MST에 edge를 추가하며 cycle이 없게 한다. 이는 union-find structure를 통해 O(logV) 로 가능
- heap을 통해 sorting없이 구현할 수도 있지만 둘다 O(nlogn)이다. 이 때문에 종종 kruskal 알고리즘은 priority queue로 구현되기도 한다.


- Wortst case O(E\*logE) 이며, 큰 graph에도 적용이 가능하다.
- edge가 이미 sort가 되었다면 알고리즘은 quasi-linear하다
- weight에 constant를 add하거나 multiply 를 하더라도 결과는 같다
- Kruskal 알고리즘은, transformation weight(add, multiply) invariant하다
- invariant하다는 것은 시스템이 어떠한 transformation에도 변화하지 않는것.


재귀적 형태 밑의 구현은 반복문형태

```python

def find(x):
    if x.parent !=x:
        x.parent = find(x)
        
    return x.parent

```

In [37]:
from functools import total_ordering

class Vertex():
    
    def __init__(self, name):
        self.name = name
        self.node = None
        
# representation of one disjoint set
class Node():
    
    def __init__(self, height, nodeId, parentNode):
        self.height = height
        self.nodeId = nodeId
        self.parentNode = parentNode
        
@total_ordering
class Edge():
    
    def __init__(self, weight, startVertex, targetVertex):
        
        self.weight = weight
        self.startVertex = startVertex
        self.targetVertex = targetVertex
        
    def __lt__(self, other):
        return self.weight < other.weight
    
    def __eq__(self, other):
        return self.weight == other.weight
    
class DisjointSet():
    
    def __init__(self, vertexList):
        self.vertexList = vertexList
        self.rootNode = []
        self.nodeCount = 0
        self.setCount = 0
        self.makeSets(vertexList)
        
    def find(self, node):
        
        currentNode = node
        
        # find parentNode
        while currentNode.parentNode is not None:
            currentNode = currentNode.parentNode
        
        root = currentNode
        currentNode = node 
        
        while currentNode is not root:
            temp = currentNode.parentNode
            currentNode.parentNode = root
            currentNode = temp
            
        return root.nodeId
    
    def merge(self, node1, node2):
        
        index1 = self.find(node1)
        index2 = self.find(node2)
        
        if index1 == index2:            
            return # they are in the same set
        
        root1 = self.rootNode[index1]
        root2 = self.rootNode[index2]
        
        if root1.height < root2.height:
            root1.parentNode = root2
            self.setCount -= 1
            
        elif root1.height > root2.height:
            root2.parentNode = root1
            self.setCount -= 1
            
        else:
            root2.parentNode = root1
            root1.height += 1
            self.setCount -= 1

    def makeSets(self, vertexList):
        
        for v in self.vertexList:
            self.makeSet(v)
            
    def makeSet(self, vertex):
        node = Node(0, len(self.rootNode), None)
        vertex.node = node
        self.rootNode.append(node)
        self.setCount += 1
        self.nodeCount += 1
        
class KruskalAlgorithm():
    
    def spanningTree(self, vertexList, edgeList):
        
        disjointSet = DisjointSet(vertexList)
        spanningTree = []
        
        edgeList.sort()
        
        for edge in edgeList:
            u = edge.startVertex
            v= edge.targetVertex
            
            if disjointSet.find(u.node) is not disjointSet.find(v.node): # not same instance
                spanningTree.append(edge)
                disjointSet.merge(u.node, v.node)
        
        for edge in spanningTree:
            print(edge.startVertex.name, " - ", edge.targetVertex.name)


In [38]:
vertex1 = Vertex("A");
vertex2 = Vertex("B");
vertex3 = Vertex("C");
vertex4 = Vertex("D");
vertex5 = Vertex("E");
vertex6 = Vertex("F");
vertex7 = Vertex("G");


edge1 = Edge(2,vertex1,vertex2);
edge2 = Edge(6,vertex1,vertex3);
edge3 = Edge(5,vertex1,vertex5);
edge4 = Edge(10,vertex1,vertex6);
edge5 = Edge(3,vertex2,vertex4);
edge6 = Edge(3,vertex2,vertex5);
edge7 = Edge(1,vertex3,vertex4);
edge8 = Edge(2,vertex3,vertex6);
edge9 = Edge(5,vertex4,vertex5);
edge10 = Edge(5,vertex4,vertex7);
edge11 = Edge(5,vertex6,vertex7);

vertexList = []
for i in range(1,8):
    vertexList.append(eval("vertex"+str(i)))

edgeList = []
for i in range(1,12):
    edgeList.append(eval("edge"+str(i)))
    
algorithm = KruskalAlgorithm()
algorithm.spanningTree(vertexList, edgeList)

C  -  D
A  -  B
C  -  F
B  -  D
B  -  E
D  -  G
